### Génération des embeddings

##### 1. Charger le modèle `paraphrase-multilingual-MiniLM-L12-v2` avec Sentence Transformers

In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')